In [1]:
import os
import math
import cv2
import numpy as np
import pandas as pd
import mediapipe as mp

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(
    static_image_mode=True,
    min_detection_confidence=0.3,
    model_complexity=2
)

In [3]:
def detectPose(image, pose):
    """
    Run MediaPipe pose on an image and return:
        output_image: image with landmarks drawn
        landmarks: list of (x, y, z) in pixel space.
    """
    output_image = image.copy()
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(imageRGB)

    height, width, _ = image.shape
    landmarks = []

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            image=output_image,
            landmark_list=results.pose_landmarks,
            connections=mp_pose.POSE_CONNECTIONS
        )
        for lm in results.pose_landmarks.landmark:
            landmarks.append((
                int(lm.x * width),
                int(lm.y * height),
                lm.z * width
            ))

    return output_image, landmarks


def calculateAngle(landmark1, landmark2, landmark3):
    """
    Angle between three keypoints (in 2D) at landmark2.
    """
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    angle = math.degrees(
        math.atan2(y3 - y2, x3 - x2) -
        math.atan2(y1 - y2, x1 - x2)
    )

    if angle < 0:
        angle += 360
    return angle

def angles_finder(landmarks):
    """
    Compute a set of angles from a list of landmarks.
    Returns a list of 14 angles in a fixed order.
    """
    # for brevity, alias the enum
    P = mp_pose.PoseLandmark

    left_elbow_angle = calculateAngle(
        landmarks[P.LEFT_SHOULDER.value],
        landmarks[P.LEFT_ELBOW.value],
        landmarks[P.LEFT_WRIST.value]
    )
    right_elbow_angle = calculateAngle(
        landmarks[P.RIGHT_SHOULDER.value],
        landmarks[P.RIGHT_ELBOW.value],
        landmarks[P.RIGHT_WRIST.value]
    )

    left_shoulder_angle = calculateAngle(
        landmarks[P.LEFT_ELBOW.value],
        landmarks[P.LEFT_SHOULDER.value],
        landmarks[P.LEFT_HIP.value]
    )
    right_shoulder_angle = calculateAngle(
        landmarks[P.RIGHT_HIP.value],
        landmarks[P.RIGHT_SHOULDER.value],
        landmarks[P.RIGHT_ELBOW.value]
    )

    left_knee_angle = calculateAngle(
        landmarks[P.LEFT_HIP.value],
        landmarks[P.LEFT_KNEE.value],
        landmarks[P.LEFT_ANKLE.value]
    )
    right_knee_angle = calculateAngle(
        landmarks[P.RIGHT_HIP.value],
        landmarks[P.RIGHT_KNEE.value],
        landmarks[P.RIGHT_ANKLE.value]
    )

    angle_for_ardhaChandrasana1 = calculateAngle(
        landmarks[P.RIGHT_ANKLE.value],
        landmarks[P.RIGHT_HIP.value],
        landmarks[P.LEFT_ANKLE.value]
    )
    angle_for_ardhaChandrasana2 = calculateAngle(
        landmarks[P.LEFT_ANKLE.value],
        landmarks[P.LEFT_HIP.value],
        landmarks[P.RIGHT_ANKLE.value]
    )

    hand_angle = calculateAngle(
        landmarks[P.LEFT_ELBOW.value],
        landmarks[P.RIGHT_SHOULDER.value],
        landmarks[P.RIGHT_ELBOW.value]
    )

    left_hip_angle = calculateAngle(
        landmarks[P.LEFT_SHOULDER.value],
        landmarks[P.LEFT_HIP.value],
        landmarks[P.LEFT_KNEE.value]
    )
    right_hip_angle = calculateAngle(
        landmarks[P.RIGHT_SHOULDER.value],
        landmarks[P.RIGHT_HIP.value],
        landmarks[P.RIGHT_KNEE.value]
    )

    neck_angle_uk = calculateAngle(
        landmarks[P.NOSE.value],
        landmarks[P.LEFT_SHOULDER.value],
        landmarks[P.RIGHT_SHOULDER.value]
    )

    left_wrist_angle_bk = calculateAngle(
        landmarks[P.LEFT_WRIST.value],
        landmarks[P.LEFT_HIP.value],
        landmarks[P.LEFT_ANKLE.value]
    )
    right_wrist_angle_bk = calculateAngle(
        landmarks[P.RIGHT_WRIST.value],
        landmarks[P.RIGHT_HIP.value],
        landmarks[P.RIGHT_ANKLE.value]
    )

    return [
        left_elbow_angle, right_elbow_angle,
        left_shoulder_angle, right_shoulder_angle,
        left_knee_angle, right_knee_angle,
        angle_for_ardhaChandrasana1, angle_for_ardhaChandrasana2,
        hand_angle,
        left_hip_angle, right_hip_angle,
        neck_angle_uk,
        left_wrist_angle_bk, right_wrist_angle_bk
    ]

In [ ]:
POSES = ["Downdog", "Goddess", "Plank", "Tree", "Warrior2"]
for pose_name in POSES:
    path = f"data/{pose_name}/Images"

    columns = [
        "pose_label",
        "image_path",
        "left_elbow_angle", "right_elbow_angle",
        "left_shoulder_angle", "right_shoulder_angle",
        "left_knee_angle", "right_knee_angle",
        "angle_for_ardhaChandrasana1", "angle_for_ardhaChandrasana2",
        "hand_angle",
        "left_hip_angle", "right_hip_angle",
        "neck_angle_uk",
        "left_wrist_angle_bk", "right_wrist_angle_bk"
    ]

    df = pd.DataFrame(columns=columns)

    for filename in os.listdir(path):
        if not (filename.lower().endswith(".jpg")
                or filename.lower().endswith(".jpeg")
                or filename.lower().endswith(".png")):
            continue

        img_path = os.path.join(path, filename)
        image = cv2.imread(img_path)
        if image is None:
            print(f"[WARN] Could not read {img_path}")
            continue

        output_image, landmarks = detectPose(image, pose)
        if landmarks:
            angles = angles_finder(landmarks)
            row = {
                "pose_label": pose_name,
                "image_path": img_path,
                "left_elbow_angle": angles[0],
                "right_elbow_angle": angles[1],
                "left_shoulder_angle": angles[2],
                "right_shoulder_angle": angles[3],
                "left_knee_angle": angles[4],
                "right_knee_angle": angles[5],
                "angle_for_ardhaChandrasana1": angles[6],
                "angle_for_ardhaChandrasana2": angles[7],
                "hand_angle": angles[8],
                "left_hip_angle": angles[9],
                "right_hip_angle": angles[10],
                "neck_angle_uk": angles[11],
                "left_wrist_angle_bk": angles[12],
                "right_wrist_angle_bk": angles[13],
            }
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)

    print(df.head())

    os.makedirs("angles", exist_ok=True)
    out_csv = f"angles/{pose_name}.csv"
    df.to_csv(out_csv, index=False)
    print("Saved angles CSV to:", out_csv)

C:\Users\sandy\AppData\Local\Temp\ipykernel_23296\896413000.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


  pose_label                        image_path  left_elbow_angle  \
0    Downdog  data/Downdog/Images\00000000.jpg        193.939749   
1    Downdog  data/Downdog/Images\00000001.jpg        160.523425   
2    Downdog  data/Downdog/Images\00000002.jpg        168.929797   
3    Downdog  data/Downdog/Images\00000003.jpg        175.207966   
4    Downdog  data/Downdog/Images\00000004.jpg        196.852595   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         199.042214           183.262344            175.340034   
1         165.983325           181.218875            178.150074   
2         166.998650           169.650461            190.287842   
3         173.291759           188.914927            170.921307   
4         184.687954           194.188427            171.774849   

   left_knee_angle  right_knee_angle  angle_for_ardhaChandrasana1  \
0       186.249932        187.856616                   358.892513   
1       181.446741        185.428325              

C:\Users\sandy\AppData\Local\Temp\ipykernel_23296\896413000.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


  pose_label                        image_path  left_elbow_angle  \
0    Goddess  data/Goddess/Images\00000000.jpg         92.528513   
1    Goddess  data/Goddess/Images\00000001.jpg         90.198943   
2    Goddess  data/Goddess/Images\00000002.jpg        112.259191   
3    Goddess  data/Goddess/Images\00000003.jpg        275.915786   
4    Goddess  data/Goddess/Images\00000004.jpg        252.360775   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         270.000000            93.780934             97.135915   
1         267.538333            94.341905             95.569443   
2         251.153282            88.434931             91.369270   
3          77.005383           271.066226            275.327813   
4         108.434949           270.125373            270.125373   

   left_knee_angle  right_knee_angle  angle_for_ardhaChandrasana1  \
0       253.610460        102.913337                   274.124119   
1       255.529706        101.103833              

C:\Users\sandy\AppData\Local\Temp\ipykernel_23296\896413000.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


  pose_label                      image_path  left_elbow_angle  \
0      Plank  data/Plank/Images\00000000.jpg        189.724648   
1      Plank  data/Plank/Images\00000001.jpg        183.366461   
2      Plank  data/Plank/Images\00000002.jpg         78.889183   
3      Plank  data/Plank/Images\00000004.jpg        170.842681   
4      Plank  data/Plank/Images\00000005.jpg        188.835113   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         187.261401           283.542549             76.546765   
1         188.495467           282.650326             79.673857   
2          74.115683            68.114973            291.989351   
3         166.792408            62.487997            296.876862   
4         195.639524           269.791568             82.730565   

   left_knee_angle  right_knee_angle  angle_for_ardhaChandrasana1  \
0       174.882685        176.879944                     0.458168   
1       177.429338        184.465135                   359.813

C:\Users\sandy\AppData\Local\Temp\ipykernel_23296\896413000.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


  pose_label                     image_path  left_elbow_angle  \
0       Tree  data/Tree/Images\00000000.jpg        324.357895   
1       Tree  data/Tree/Images\00000001.jpg        320.977140   
2       Tree  data/Tree/Images\00000002.jpg        248.702646   
3       Tree  data/Tree/Images\00000003.jpg        163.750852   
4       Tree  data/Tree/Images\00000004.jpg        311.484397   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0          34.517054            25.373335             16.675563   
1          34.053269            37.973141             31.524500   
2         105.966831            53.447527             47.563770   
3         187.967541           158.726263            157.045285   
4          49.472960            35.818011             24.227745   

   left_knee_angle  right_knee_angle  angle_for_ardhaChandrasana1  \
0       186.628271         24.145542                    20.947471   
1       180.333758         26.337688                     8.540115   

C:\Users\sandy\AppData\Local\Temp\ipykernel_23296\896413000.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


  pose_label                         image_path  left_elbow_angle  \
0   Warrior2  data/Warrior2/Images\00000000.jpg        170.344221   
1   Warrior2  data/Warrior2/Images\00000001.jpg        183.660993   
2   Warrior2  data/Warrior2/Images\00000002.jpg        174.981235   
3   Warrior2  data/Warrior2/Images\00000004.jpg        179.608575   
4   Warrior2  data/Warrior2/Images\00000005.jpg        188.620162   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         190.238198            94.800976             84.995013   
1         178.417646           109.447136             99.541716   
2         189.430824            86.557785             95.401143   
3         185.095398           108.674743             96.636949   
4         176.905942           111.763599             96.188116   

   left_knee_angle  right_knee_angle  angle_for_ardhaChandrasana1  \
0       262.092837        185.470796                   249.802607   
1       185.548886        110.599738        

In [9]:
# merge all csv files into one
RESULTS_DIR = "angles/"
all_dfs = []

for pose_name in POSES:
    csv_path = os.path.join(RESULTS_DIR, f"{pose_name}.csv")
    df = pd.read_csv(csv_path)

    if "Label" in df.columns and "image_path" not in df.columns:
        df = df.rename(columns={"Label": "image_path"})

    all_dfs.append(df)

# concatenate
df_all = pd.concat(all_dfs, ignore_index=True)
print(df_all.head())
print("Shape:", df_all.shape)

out_csv = os.path.join(RESULTS_DIR, "all_angles.csv")
df_all.to_csv(out_csv, index=False)
print("Saved merged angles to:", out_csv)

  pose_label                        image_path  left_elbow_angle  \
0    Downdog  data/Downdog/Images\00000000.jpg        193.939749   
1    Downdog  data/Downdog/Images\00000001.jpg        160.523425   
2    Downdog  data/Downdog/Images\00000002.jpg        168.929797   
3    Downdog  data/Downdog/Images\00000003.jpg        175.207966   
4    Downdog  data/Downdog/Images\00000004.jpg        196.852595   

   right_elbow_angle  left_shoulder_angle  right_shoulder_angle  \
0         199.042214           183.262344            175.340034   
1         165.983325           181.218875            178.150074   
2         166.998650           169.650461            190.287842   
3         173.291759           188.914927            170.921307   
4         184.687954           194.188427            171.774849   

   left_knee_angle  right_knee_angle  angle_for_ardhaChandrasana1  \
0       186.249932        187.856616                   358.892513   
1       181.446741        185.428325              